# Observação e preparação

Material baseado no trabalho desenvolvido pelo Estadão - Como votou sua vizinhança https://github.com/estadao/como-votou-sua-vizinhanca

In [3]:
#Bibliotecas

# Acesso ao arquivos do Google Drive
# Na dúvida sobre o caminho para os arquivos, no menu da lateral esquerda, 
# clique no icone em formato de pasta, 
# navegue até a pasta ou arquivo desejado, ao clicar com o botação direito do mouse
# é possivel copiar o caminho
from google.colab import drive
drive.mount('/content/gdrive')

#Outras
import pandas as pd

Mounted at /content/gdrive


# Base de dados dos votos

Os dados podem ser obtidos no repositório de dados do TSE, e são disponibilizados por estado e que podem ser pesados. O arquivo com os votos do 1o turno na eleição de 2018 de São Paulo possui 4GB (não zipado).

https://dadosabertos.tse.jus.br/dataset/

# Carregando a base de dados

Atenção para as colunas QT_APTOS, QT_COMPARECIMENTO, QT_ABSTENÇÔES: esses são os números da urna. Cuidado para não soma-los ao fazer algum tipo de agregação.

In [1]:
dtype = {'ANO_ELEICAO':'str',
          'NR_TURNO':'str',
          'SG_UF':'str',
          'CD_MUNICIPIO':'str',
          'NM_MUNICIPIO':'str',
          'NR_ZONA':'str',
          'NR_SECAO':'str',
          'NR_LOCAL_VOTACAO':'str',
          'CD_CARGO_PERGUNTA':'str',
          'DS_CARGO_PERGUNTA':'str',
          'NR_PARTIDO':'str',
          'SG_PARTIDO':'str',
          'NM_PARTIDO': 'str',
          'QT_APTOS':'str',
          'QT_COMPARECIMENTO':'str',
          'QT_ABSTENCOES':'str',
          'CD_TIPO_VOTAVEL':'str',
          'DS_TIPO_VOTAVEL':'str',
          'NR_VOTAVEL':'str',
          'NM_VOTAVEL':'str',
          'QT_VOTOS':'int'}

usecols = [
    'ANO_ELEICAO', 'NR_TURNO',
    'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
    'CD_CARGO_PERGUNTA', 'DS_CARGO_PERGUNTA', 
    'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO',
    'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES', 
    'CD_TIPO_VOTAVEL', 'DS_TIPO_VOTAVEL', 'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS'
]

In [2]:
%%time
votos_geral = pd.read_csv("/content/gdrive/MyDrive/dados/data/votos/boletim-urna-original/bweb_1t_SP_181120201549.csv",
                              encoding='Latin1', sep=';', header=0, 
                              dtype=dtype,
                              usecols=usecols
                             )

NameError: ignored

In [ ]:
#Criando o local_unico - chave que permite cruzar com a base dos locais
votos_geral['local_unico'] = votos_geral.CD_MUNICIPIO + '-' + votos_geral.NR_ZONA + '-' + votos_geral.NR_LOCAL_VOTACAO

In [ ]:
#Visão geral da base
votos_geral.head(3)

In [ ]:
# Consolidar os votos pelo local de votação, eliminando a seção eleitoral (NR_SECAO)
# Também são desconsideradas as colunas QT_APTOS, QT_COMPARECIMENTO, QT_ABSTENÇÔES:
votos_geral_consolidado = pd.pivot_table(votos_geral, index=['ANO_ELEICAO', 'NR_TURNO', 'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO',
       'NR_ZONA', 'NR_LOCAL_VOTACAO', 
       'CD_CARGO_PERGUNTA', 'DS_CARGO_PERGUNTA', 
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'CD_TIPO_VOTAVEL', 'DS_TIPO_VOTAVEL', 'NR_VOTAVEL', 'NM_VOTAVEL', 
       'local_unico'],
       values=['QT_VOTOS'], aggfunc='sum', fill_value=0).reset_index()

In [ ]:
votos_geral_consolidado

,ANO_ELEICAO,NR_TURNO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,local_unico,QT_VOTOS
0,2020,1,SP,61000,EUCLIDES DA CUNHA PAULISTA,330,1031,11,Prefeito,-1,#NULO#,#NULO#,2,Branco,95,Branco,61000-330-1031,46
1,2020,1,SP,61000,EUCLIDES DA CUNHA PAULISTA,330,1031,11,Prefeito,-1,#NULO#,#NULO#,3,Nulo,96,Nulo,61000-330-1031,141
2,2020,1,SP,61000,EUCLIDES DA CUNHA PAULISTA,330,1031,11,Prefeito,10,REPUBLICANOS,REPUBLICANOS,1,Nominal,10,NENE LOPES,61000-330-1031,1772
3,2020,1,SP,61000,EUCLIDES DA CUNHA PAULISTA,330,1031,11,Prefeito,45,PSDB,Partido da Social Democracia Brasileira,1,Nominal,45,EDMILSON,61000-330-1031,646
4,2020,1,SP,61000,EUCLIDES DA CUNHA PAULISTA,330,1031,13,Vereador,-1,#NULO#,#NULO#,2,Branco,95,Branco,61000-330-1031,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2610289,2020,1,SP,72737,VARGEM GRANDE PAULISTA,286,1228,13,Vereador,55,PSD,Partido Social Democrático,1,Nominal,55523,BECA,72737-286-1228,13
2610290,2020,1,SP,72737,VARGEM GRANDE PAULISTA,286,1228,13,Vereador,55,PSD,Partido Social Democrático,1,Nominal,55630,MAURO DA OFICINA,72737-286-1228,39
2610291,2020,1,SP,72737,VARGEM GRANDE PAULISTA,286,1228,13,Vereador,55,PSD,Partido Social Democrático,1,Nominal,55640,ROBERTÃO,72737-286-1228,2
2610292,2020,1,SP,72737,VARGEM GRANDE PAULISTA,286,1228,13,Vereador,55,PSD,Partido Social Democrático,1,Nominal,55677,DUDA GODOY,72737-286-1228,2


In [ ]:
votos_geral_consolidado.to_csv('/content/gdrive/MyDrive/dados/data/votos/boletim-urna-consolidado/2020_SP_consolidado.csv', sep=';', index=False)

In [ ]:
#Selecionar uma parte da base
votos_selecao = votos_geral_consolidado[(votos_geral_consolidado['CD_MUNICIPIO'] == '71072')]

In [ ]:
votos_selecao

,ANO_ELEICAO,NR_TURNO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,local_unico,QT_VOTOS
1577092,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,-1,#NULO#,#NULO#,2,Branco,95,Branco,71072-1-1015,134
1577093,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,-1,#NULO#,#NULO#,3,Nulo,96,Nulo,71072-1-1015,221
1577094,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,10,REPUBLICANOS,REPUBLICANOS,1,Nominal,10,CELSO RUSSOMANNO,71072-1-1015,381
1577095,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,13,PT,Partido dos Trabalhadores,1,Nominal,13,JILMAR TATTO,71072-1-1015,115
1577096,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,16,PSTU,Partido Socialista dos Trabalhadores Unificado,1,Nominal,16,VERA,71072-1-1015,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395164,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90038,MAJOR HORTA,71072-6-2097,1
2395165,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90123,NENEM,71072-6-2097,1
2395166,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90444,ROBSON TRINDADE,71072-6-2097,1
2395167,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90600,RAFINHA,71072-6-2097,1


In [ ]:
votos_selecao.to_csv('/content/gdrive/MyDrive/dados/data/votos/boletim-urna-consolidado/2020_71110_geral.csv', index=False, sep=';')

In [ ]:
## OPÇÃO 
#Agregar os votos por partidos
votos_consolidado_partido = pd.pivot_table(votos_geral, index=['ANO_ELEICAO', 'NR_TURNO', 'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO',
       'NR_ZONA', 'NR_LOCAL_VOTACAO', 'CD_CARGO_PERGUNTA',
       'DS_CARGO_PERGUNTA', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'local_unico'],  
       values=['QT_VOTOS'], aggfunc='sum', fill_value=0).reset_index()

In [ ]:
votos_consolidado_partido

,ANO_ELEICAO,NR_TURNO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS
0,2020,1,SP,71072,SÃO PAULO,1,1,1015,13,Vereador,-1,#NULO#,#NULO#,386,241,145,19
1,2020,1,SP,71072,SÃO PAULO,1,1,1015,13,Vereador,10,REPUBLICANOS,REPUBLICANOS,386,241,145,10
2,2020,1,SP,71072,SÃO PAULO,1,1,1015,13,Vereador,11,PP,PROGRESSISTAS,386,241,145,8
3,2020,1,SP,71072,SÃO PAULO,1,1,1015,13,Vereador,12,PDT,Partido Democrático Trabalhista,386,241,145,2
4,2020,1,SP,71072,SÃO PAULO,1,1,1015,13,Vereador,13,PT,Partido dos Trabalhadores,386,241,145,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582327,2020,1,SP,71072,SÃO PAULO,6,99,1074,13,Vereador,50,PSOL,Partido Socialismo e Liberdade,419,266,153,35
582328,2020,1,SP,71072,SÃO PAULO,6,99,1074,13,Vereador,51,PATRIOTA,Patriota,419,266,153,13
582329,2020,1,SP,71072,SÃO PAULO,6,99,1074,13,Vereador,55,PSD,Partido Social Democrático,419,266,153,14
582330,2020,1,SP,71072,SÃO PAULO,6,99,1074,13,Vereador,65,PC do B,Partido Comunista do Brasil,419,266,153,4


In [ ]:
votos_consolidado_partido.to_csv('/content/gdrive/MyDrive/dados/data/votos/boletim-urna-consolidado/2020_71110_partido.csv', index=False)